# 5-박주홍
## Setting

라이브러리를 로드합니다. 주요 라이브러리로 numpy와 sklearn을 사용합니다.

In [3]:
# import required modules

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

## Function
분석에 사용할 함수를 정의합니다. 함수의 역할에 대해서는 아래 한글 주석으로 추가하였습니다.

In [4]:
# 모델 학습 결과를 다양한 Metric으로 평가하는 함수, 아래 주석을 통해 평가하는 13가지 Metric에 대해 설명하고 있음
def calculate_diagnostic_performance (actual_predicted):
    """ Calculate diagnostic performance.
    
    Takes a Numpy array of 1 and zero, two columns: actual and predicted
    
    Note that some statistics are repeats with different names
    (precision = positive_predictive_value and recall = sensitivity).
    Both names are returned
    
    Returns a dictionary of results:
        
    1) accuracy: proportion of test results that are correct    
    2) sensitivity: proportion of true +ve identified
    3) specificity: proportion of true -ve identified
    4) positive likelihood: increased probability of true +ve if test +ve
    5) negative likelihood: reduced probability of true +ve if test -ve
    6) false positive rate: proportion of false +ves in true -ve patients
    7) false negative rate:  proportion of false -ves in true +ve patients
    8) positive predictive value: chance of true +ve if test +ve
    9) negative predictive value: chance of true -ve if test -ve
    10) precision = positive predictive value 
    11) recall = sensitivity
    12) f1 = (2 * precision * recall) / (precision + recall)
    13) positive rate = rate of true +ve (not strictly a performance measure)
    """
    # Calculate results
    actual_positives = actual_predicted[:, 0] == 1
    actual_negatives = actual_predicted[:, 0] == 0
    test_positives = actual_predicted[:, 1] == 1
    test_negatives = actual_predicted[:, 1] == 0
    test_correct = actual_predicted[:, 0] == actual_predicted[:, 1]
    accuracy = np.average(test_correct)
    true_positives = actual_positives & test_positives
    true_negatives = actual_negatives & test_negatives
    sensitivity = np.sum(true_positives) / np.sum(actual_positives)
    specificity = np.sum(true_negatives) / np.sum(actual_negatives)
    positive_likelihood = sensitivity / (1 - specificity)
    negative_likelihood = (1 - sensitivity) / specificity
    false_positive_rate = 1 - specificity
    false_negative_rate = 1 - sensitivity
    positive_predictive_value = np.sum(true_positives) / np.sum(test_positives)
    negative_predictive_value = np.sum(true_negatives) / np.sum(test_negatives)
    precision = positive_predictive_value
    recall = sensitivity
    f1 = (2 * precision * recall) / (precision + recall)
    positive_rate = np.mean(actual_predicted[:,1])
    
    # Add results to dictionary
    performance = {}
    performance['accuracy'] = accuracy
    performance['sensitivity'] = sensitivity
    performance['specificity'] = specificity
    performance['positive_likelihood'] = positive_likelihood
    performance['negative_likelihood'] = negative_likelihood
    performance['false_positive_rate'] = false_positive_rate
    performance['false_negative_rate'] = false_negative_rate
    performance['positive_predictive_value'] = positive_predictive_value
    performance['negative_predictive_value'] = negative_predictive_value
    performance['precision'] = precision
    performance['recall'] = recall
    performance['f1'] = f1
    performance['positive_rate'] = positive_rate

    return performance

# 데이터를 로드하는 함수, 주석으로 데이터에 대한 설명이 추가됨
def load_data ():
    """Load the data set. Here we load the Breast Cancer Wisconsin (Diagnostic)
    Data Set. Data could be loaded from other sources though the structure
    should be compatible with thi sdata set, that is an object with the 
    following attribtes:
        .data (holds feature data)
        .feature_names (holds feature titles)
        .target_names (holds outcome classification names)
        .target (holds classification as zero-based number)
        .DESCR (holds text-based description of data set)"""
    
    data_set = datasets.load_breast_cancer()
    return data_set

# 학습을 안정적으로 하기 위해 feature의 scale을 보정하여 normalise하는 코드
def normalise (X_train,X_test):
    """Normalise X data, so that training set has mean of zero and standard
    deviation of one"""
    
    # Initialise a new scaling object for normalising input data
    sc=StandardScaler()
    
    # Set up the scaler just on the training set
    sc.fit(X_train)
    
    # Apply the scaler to the training and test sets
    X_train_std=sc.transform(X_train)
    X_test_std=sc.transform(X_test)
    
    return X_train_std, X_test_std

# 학습 결과를 보기 편하게 출력하는 함수
def print_diagnostic_results (performance):
    """Iterate through, and print, the performance metrics dictionary"""
    
    print('\nMachine learning diagnostic performance measures:')
    print('-------------------------------------------------')
    for key, value in performance.items():
        print (key,'= %0.3f' %value) # print 3 decimal places
    return

# 학습과 평가에 사용하기 위해 데이터를 X, y 로 나누고 이를 Train 및 Test로 나누하는 함수
def split_data (data_set, split=0.25):
    """Extract X and y data from data_set object, and split into tarining and
    test data. Split defaults to 75% training, 25% test if not other value 
    passed to function"""
    
    X=data_set.data
    y=data_set.target
    X_train,X_test,y_train,y_test=train_test_split(
        X,y,test_size=split, random_state=0)
    return X_train,X_test,y_train,y_test

# 모델로 Test를 하는 함수, 입력값에 대한 예측값을 출력함
def test_model(model, X, y):
    """Return predicted y given X (attributes)"""
    
    y_pred = model.predict(X)
    test_results = np.vstack((y, y_pred)).T
    return test_results

# 모델을 Train 하는 함수, 학습된 모델을 출력함
# Reproducible한 결과를 위해 random_state는 0으로 지정
# 학습을 위해 solver로 Limited-memory BFGS를 사용함
# 모델 구조는 hidden layer (50,5) 으로 하였고 activation으로 relu를 사용하였음
def train_model (X, y):
    """Train the model """
    from sklearn.neural_network import MLPClassifier
    model = MLPClassifier(solver='lbfgs', alpha=1e-8, hidden_layer_sizes=(50, 5),
                        max_iter=100000, shuffle=True, learning_rate_init=0.001,
                        activation='relu', learning_rate='constant', tol=1e-7,
                        random_state=0)
    model.fit(X_train_std, y_train)   
    return model

## Analysis
실제 분석을 하는 항목입니다.

In [5]:
###### Main code #######

# 데이터를 로드
# Load data
data_set = load_data()

# 데이터를 X, y에 대해 train, test로 나눔
# Split data into trainign and test sets
X_train, X_test, y_train, y_test = split_data(data_set, 0.25)

# 안정적인 학습을 위해 Feature들의 sclae 차이를 보정하는 Normalize 수행
# Normalise data
X_train_std, X_test_std = normalise(X_train,X_test)

# 모델을 학습
# Train model
model = train_model(X_train_std,y_train)

# Test 데이터에 대한 모델 예측 결과를 생성
# Produce results for test set
test_results = test_model(model, X_test_std, y_test)

# Test 데이터에 대한 모델 예측 결과를 바탕으로 각종 Metric 생성
# Measure performance of test set predictions
performance = calculate_diagnostic_performance(test_results)

# 위에서 생성된 모델 Metric을 출력
# Print performance metrics
print_diagnostic_results(performance)


Machine learning diagnostic performance measures:
-------------------------------------------------
accuracy = 0.958
sensitivity = 0.967
specificity = 0.943
positive_likelihood = 17.078
negative_likelihood = 0.035
false_positive_rate = 0.057
false_negative_rate = 0.033
positive_predictive_value = 0.967
negative_predictive_value = 0.943
precision = 0.967
recall = 0.967
f1 = 0.967
positive_rate = 0.629
